## Batting Pitching Fielding

After my last notebook, Moneyball, I came to wonder in what areas in baseball are statistically important to winning. Thus, I decided to look at the games in three main categories: batting, pitching, and fielding.

    Question : From batting, pitching and fielding, what's most important in winning baseball?

In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns',None)

In [118]:
df = pd.read_csv('/Users/jinc/Desktop/프로젝트/baseball_analysis/baseball_analysis/lahman/core/Teams.csv')
df_pitch = pd.read_csv('/Users/jinc/Desktop/프로젝트/baseball_analysis/baseball_analysis/lahman/core/Pitching.csv')
df_field = pd.read_csv('/Users/jinc/Desktop/프로젝트/baseball_analysis/baseball_analysis/fangraph_field.csv')
df_w = pd.read_csv('/Users/jinc/Desktop/프로젝트/baseball_analysis/baseball_analysis/lahman/core/FanGraphs Leaderboard.csv')

## Feature Selection

#### **Pitching**

There are various stats that you can use to evaluate pitcher's performance, such as ERA, WHIP, FIP... In this analysis, we will be using FIP stats because it focuses soley on the events a pitcher has the most control over. FIP are intended as a measure of a pitcher's performance unaffected by the defense behind him.

-Pure Pitching skills!!

#### **Batting**

With batting, we have stats like OPS, OPS+, OBP... We will be using wOBA stat, which is the stat that is favored by many sabermetricians because of its accuracy and simplicity. wOBA is a version of on-base percentage that accounts for 'how' a player reached base instead of simply consedering whether a player reached base. The value for each base is determined by how much that event is worth in relation to projected run scored. Since hitter's main objective is to score runs, I thought that this would be a good stat to start with.

#### **Fielding**

Fielding is quite recent in baseball statistics. There are great indicators like UZR and DRS, but the data only exist from 2003. Thus, we need to find other indicator that can represent older baseballs. In this notebook, we will use FP(Fielding Percentage) to evaluate teams' fielding performance. 

FP is a measure that reflect the percentage of times a defensive player properly handles a batted or thrown ball. 

-------


For more information click [here](https://www.crawfishboxes.com/2015/4/30/8519577/sabermetrics-10-best-stats-more)

## Data Cleaning

In [119]:
pitch_hbp = df_pitch.groupby(['yearID', 'teamID'])['HBP'].sum().to_frame().reset_index()
pitch_hbp.rename(columns={'HBP':'PHBP'}, inplace=True)

df = df.merge(pitch_hbp, how='left',on=['yearID','teamID'])
df = df.merge(df_w, how='left', left_on='yearID', right_on='Season')

# 1B = H - 2B - 3B - HR
df['1B'] = df.H - df['2B'] - df['3B'] - df.HR

# FIP = 13*HR + 3*(BB+HBP) - 2*SO)/(IPouts) + cFIP
df['FIP'] = ((df.HRA*13 + (df.BBA+df.PHBP)*3 - df.SOA*2)) / df.IPouts + df.cFIP

# wOBA = wBB*(BB-IBB) + wHBP*(HBP) + w1B*(1B) + w2B*(2B) + w3B*(3B) + wHR*(HR) / (AB+BB-IBB+SF+HBP)
df['wOBA_team'] = ((df.wBB*df.BB) + (df.wHBP*df.HBP) + (df.w1B*df['1B']) + (df.w2B*df['2B']) + (df.w3B*df['3B']) + (df.wHR*df.HR)) / (df.AB+df.BB+df.SF+df.HBP)

In [ ]:
df.merge()